# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Notebook Principal

**Grupo 30: Datatouille**

- 101055 - Bojman, Camila
- 100029 - del Mazo, Federico
- 100687 - Hortas, Cecilia
- 97649 - Souto, Rodrigo

**http://fdelmazo.github.io/7506-Datos/**

**https://www.kaggle.com/datatouille2018/competitions**

Continuando la investigación sobre la empresa Trocafone realizada en el [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html), se busca determinar la probabilidad de que un usuario del sitio realice una conversión en el período determinado.

Notebooks en orden de corrida y lectura:

0. [TP1](https://fdelmazo.github.io/7506-Datos/TP1/TP1.html) --> Familiarización con el set de datos y exploración de estos.

1. [Investigación Previa](https://fdelmazo.github.io/7506-Datos/TP2/investigacion.html) --> Con ayuda de lo trabajado en el TP1, se averiguan más cosas de las datos, en busqueda de que poder reutilizar.

2. [Creación de Dataframes](https://fdelmazo.github.io/7506-Datos/TP2/new-dataframes.html) --> Como parte del feature engineering, se crean dataframes nuevos con información de los productos del sitio y de como se accede a este (marcas, sistemas operativos, etc).

3. [Feature Engineering](https://fdelmazo.github.io/7506-Datos/TP2/feature-engineering.html) -> Busqueda de atributos de los usuarios de los cuales se busca predecir la conversión

4. Algoritmos de clasificación (directorio `AlgoritmosClasificacion`) --> Definiciones de algoritmos de machine learning para utilizar en este notebook

  * [Árboles de decisión](https://fdelmazo.github.io/7506-Datos/TP2/AlgoritmosClasificacion/decision-tree.html): TODO, descr
  * [Random Forest](https://fdelmazo.github.io/7506-Datos/TP2/AlgoritmosClasificacion/random-forest.html): TODO, descr
  * [XGBoost](https://fdelmazo.github.io/7506-Datos/TP2/AlgoritmosClasificacion/xgboost.html): TODO, descr

5. TP2 --> Teniendo todo en cuenta, juntar todos los datos buscados y encontrados sobre un mismo dataframe, aplicarle todos los algoritmos definidos previamente e ir realizando los entrenamientos y posteriores predicciones de conversiones.

In [ ]:
# Set-up inicial, se deja comentado para evitar instalarle módulos al usuario

## Primero, descargar los datasets de no tenerlos

### Antes de comenzar, buscar las credenciales de kaggle (el nombre de usuario, y el token) y setearlos en las siguientes dos lineas
### %env KAGGLE_USERNAME="datatouille2018"
### %env KAGGLE_KEY="xxx"

### !pip install kaggle # https://github.com/Kaggle/kaggle-api
### !kaggle competitions download -c trocafone -p data
### !unzip -q data/events_up_to_01062018.csv.zip -d data
### !rm data/events_up_to_01062018.csv.zip
### !ls data/

## Luego, descargar los módulos a utilizar a lo largo de todo el trabajo

### !pip install nbimporter
### !conda install -c conda-forge xgboost 

In [ ]:
import nbimporter
import pandas as pd
from sklearn.model_selection import train_test_split

from AlgoritmosClasificacion import *

In [ ]:
df_events = pd.read_csv('data/events_up_to_01062018.csv', low_memory=False)
df_users = pd.read_csv('data/user-features.csv',low_memory=False)
df_sessions = pd.read_csv('data/sessions.csv', low_memory=False)
df_brands = pd.read_csv('data/brands.csv')
df_os = pd.read_csv('data/os.csv')
df_browsers = pd.read_csv('data/browsers.csv')
df_y = pd.read_csv('data/labels_training_set.csv')
df_y = df_y.groupby('person').sum()

df = df_events.merge(df_sessions, how='left', left_index=True, right_index=True)
df = df.merge(df_browsers, how='left', on='browser_version')
df = df.merge(df_os, how='left', on='operating_system_version')
df = df.merge(df_brands, how='left', on='model')
df = df.merge(df_users)

In [ ]:
# Los atributos con pocos valores posibles se pasan a variables categoricas para ahorrar memoria
df['event'] = df['event'].astype('category')
df['condition'] = df['condition'].astype('category')
df['storage'] = df['storage'].astype('category')
df['search_engine'] = df['search_engine'].astype('category')
df['channel'] = df['channel'].astype('category')
df['device_type'] = df['device_type'].astype('category')

df['brand'] = df['brand'].astype('category')
df['operating_system'] = df['operating_system'].astype('category')
df['browser'] = df['browser'].astype('category')

# El tiempo es mejor manejarlo como tal
df['timestamp'] = pd.to_datetime(df['timestamp'])

df['month_number'] = df['timestamp'].dt.month
df['month_name'] = df['month_number'].apply(lambda x: calendar.month_abbr[x])
df['week_day'] = df['timestamp'].dt.weekday
df['week_number'] = df['timestamp'].dt.week
df['week_day_name'] = df['timestamp'].dt.weekday_name
df['day_date'] = df['timestamp'].dt.to_period('D')
df['day_dom'] = df['timestamp'].dt.day
df['day_doy'] = df['timestamp'].dt.dayofyear
df['hour_count'] = df['timestamp'].dt.hour

In [ ]:
X, y = df.iloc[:,:-1],data.iloc[:,-1]

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

## Submission Framework

In [ ]:
def require(x1, x2):
    if x1 != x2:
        print('ERROR - {} must be equal to {}'.format(str(x1), str(x2)))
        raise ValueError('Oh la la.') 

def df_label_xor(df1, df2):
    
    merged = df1.merge(df2, how='outer', left_index=True, right_index=True, indicator=True)
    merged = merged.query('_merge != "both"')
    return merged
    
# Crea la matriz X y el vector y para entrenar
def fr1_extract_X_y(df, df_y):
    require(len(df), 38829)
    require(len(df_y), 19414)
    
    data = df.merge(df_y, how='inner', left_index= True, right_index=True)
    require(len(data), 19414)
    
    X = data.drop('label', axis=1).values
    y = df_y.values
    
    return X, y

# Splitea para generar los set de entrenamiento y de prueba
def fr2_train_test_split(X, y, seed):
    X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                        test_size=0.34,
                                                        stratify=y,
                                                        random_state=seed)
    return X_train, X_test, y_train, y_test

# Customizado, se tiene que hacer uno por algoritmo
def fr3_decision_tree(X_train, X_test, y_train, seed):
    tree = DecisionTreeClassifier(criterion='gini',
                              min_samples_leaf=5,
                              min_samples_split=5,
                              max_depth=3,
                              random_state=seed)

    tree.fit(X_train, y_train)
    return tree

def fr3_random_forest(X_train, X_test, y_train, seed):
    rf = RandomForestClassifier(n_estimators=20,
                           n_jobs=2,
                           min_samples_split=300,
                           random_state=seed,
                           class_weight='balanced')

    rf.fit(X_train, y_train)
    return rf
    
def fr4_accuracy_score(X_test, y_test, model, model_name):
    y_pred = model.predict_proba(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print('{} accuracy score: {}'.format(model_name, accuracy))    
    
# Crea la matriz X a predecir
def fr5_extract_X_to_predict(df, df_y, model):
    require(len(df), 38829)
    require(len(df_y), 19414)
       
    data = df_label_xor(df, df_y)
    data = data.drop(['label', '_merge'], axis=1)

    require(len(data), 19415)

    predictions = model.predict(data.values)
    return data, predictions

# Devuelve la cantidad de 1s predecidos
def fr6_print_information(df, predictions, model):
    print('Ammount of 1s: {}'.format(predictions.sum()))
    
    feature_importances = pd.DataFrame(model.feature_importances_,
                                  index=df.columns,
                                  columns=['importance'])
    feature_importances = feature_importances.sort_values('importance', ascending=False)

    display(feature_importances)
    
    
def fr7_to_csv(df, predictions, name_csv):
    submission = df
    submission['label'] = predictions
    submission = submission['label']
    
    require(len(submission), 19415)
    
    submission.to_csv(name_csv, header=True)

### Dummy Submission

In [ ]:
# Dummys de todos 0s y todos 1s
gb = df[['person', 'week_number']].groupby('person')
dummy = gb.agg('sum')

dummy = dummy['week_number'] * 0
dummy = dummy.to_frame()
print("dummy-size: "+str(len(dummy))+" | y-size: "+str(len(df_y)))
y_df_tmp = df_y.set_index(df_y.index)[['label']]

dummy_final = dummy.merge(y_df_tmp, how='outer', left_index=True, right_index=True, indicator=True)
#display(dummy.head())
#display(y_df.head())
#display(dummy_final.head())
dummy['label'] = dummy['week_number'] * 0
dummy = dummy[['label']]
dummy = dummy[~dummy.index.isin(df_y.index)]
display(len(dummy))
dummy
display(len(dummy_final.query('_merge != "both"')))
dummy_final = dummy_final.query('_merge != "both"')[['week_number']]
dummy_final.columns = ['label']
dummy_final['label'] = dummy_final['label'] + 1
print("checking sum is zero: ")
display(dummy_final.sum())
dummy_final.head()

dummy_final.to_csv('submit-zeros.csv', header=True)

## Algoritmos de Machine Learning

---

### Decision Tree


---

### Random Forest

---

### XGBoost


In [ ]:
AlgoritmosClasificacion.xgboost.XGboost()

---
---

### Lista de robos

TODO: Borrar

* [ ] https://github.com/urielkelman/abracadata/tree/master/TP2

* [ ] https://github.com/GastonMontes/Datos-TP2

* [ ] https://github.com/MatiasReimondo/Datos

* [ ] Ver diapos argerich feature engineering

* [ ] Spammear ramos mejia de preguntas

* [ ] Notebooks de Martin

* [ ] Notebook de Juan

### Lista de cosas a hacer

* [ ] Dividir en set de entrenamientos y set de test. Ojo, hay que hacerlo con tiempo! No se puede hacer al azar. Los primeros meses entrenan a los siguientes. Etc

* [ ] Identificar bias y varianza, ploteando error de set de entrenamiento y error de set de test en funcion de cantidad de datos en set de entrenamiento (mismo plot)

* [ ] Perturbar datos de entrada -> reducir overfitting

* [ ] Feature engineering --> no olvidar documentar y versionar!!!

* [ ] Catboost

* [ ] Feature: dia de la semana

* [ ] Loopear interacciones importantes! Random forest claveee

* [ ] No perder nombre de feature original

* [ ] Scrapear trocafone --> Precios

* [ ] Xgboost no sirve para importancia de features. No es estable

* [ ] Reclamar 50 usd aws &#128544; 

* [ ] Clustering para nuevos features para entrenar